In [ ]:
# TODO 
# IMPROVE SUBJECT DF CREATION !!!!
# REMOVE NON NEEDED VISUALISATIONS !!!!!!!

In [ ]:
# Installation of BioSignalsNotebooks
# %pip install biosignalsnotebooks
# %pip install tqdm

In [ ]:
# Imports
import os
import math
import pandas as pd
import tensorflow as tf 
# import plotly.graph_objects as go
import biosignalsnotebooks as bsnb
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm  #testing this, if TQDM breaks remove it 

In [ ]:
# Function to create a dataframe to store the data of every subject across all circuits 
def subject_df_creator(subject_id):
    circuits_dfs_list = []
    output_msg=[]
    for circuit_number in tqdm( range(1, 51), desc=f"Concatenating Circuit Files for Subject AB{subject_id}"):
        filename = f"5362627/AB{subject_id}/AB{subject_id}/Raw/AB{subject_id}_Circuit_0{circuit_number:02d}_raw.csv"  
        if not os.path.exists(filename):  # Check if the file exists
            output_msg.append(f"0{circuit_number:02d}")
            continue
        df_circuit = pd.read_csv(filename)
        # Dropping the Unneeded Gyroscope and Speedometer Columns 
        df_circuit = df_circuit.drop(df_circuit.columns[0:30], axis=1)
        df_circuit = df_circuit.drop(df_circuit.columns[14:], axis=1)
        # df_circuit['Subject'] = f"AB{subject_id}"
        circuits_dfs_list.append(df_circuit)
    
    # Concatenate all DataFrames in the list along the rows axis
    merged_df = pd.concat(circuits_dfs_list, ignore_index=True)
    if output_msg: 
        print(f"{len(output_msg)} Files do not exist:", output_msg)
    return merged_df

In [ ]:
# Defining important lists
subjects = ["156", "185", "186", "188", "189", "190", "191", "192", "193", "194"]
muscles = ['Right Tibialis Ant.', 'Left Tibialis Ant.', 'Right MG', 'Left MG', 'Right SOL', 'Left SOL', 'Right BF', 'Left BF', 'Right ST', 'Left ST', 'Right VL', 'Left VL', 'Right RF', 'Left RF']

In [ ]:
# Creating dataframes to save subject-specific data
df_subject_156 = subject_df_creator("156")
df_subject_185 = subject_df_creator("185")
df_subject_186 = subject_df_creator("186")
df_subject_188 = subject_df_creator("188")
df_subject_189 = subject_df_creator("189")
df_subject_190 = subject_df_creator("190")
df_subject_191 = subject_df_creator("191")
df_subject_192 = subject_df_creator("192")
df_subject_193 = subject_df_creator("193")
df_subject_194 = subject_df_creator("194")
# IMPROVE !!!!!!!!!!!!!!!!!!!!!!

In [ ]:
# Merging all the subject dataframes into one
list_of_all_subjects_dfs = [df_subject_156, df_subject_185, df_subject_186 , df_subject_188, df_subject_189, df_subject_190, df_subject_191, df_subject_192, df_subject_193, df_subject_194]
df_all_subjects = pd.concat(list_of_all_subjects_dfs, ignore_index=True)

In [ ]:
df_all_subjects[:10]

# Visualisation 

In [ ]:
# # adding figures and traces
# fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x=df_all_subjects.index/1000, y=df_all_subjects['Right_TA'][60000:90000]))
# fig1.update_layout( title="sEMG Signal: Sitting Vs Contraction Bursts Vs Rest", xaxis_title="Time (s)",
#                     yaxis_title="sEMG Activity (V)", margin=dict(l=50, r=50, b=50, t=50, pad=4),
#                     autosize=False, width=800, height=300)
# # plotting
# fig1.show()

## Analysis

In [ ]:
# Studying mean, sigma and variance of the 14 Muscles
df_analysis = pd.DataFrame()
df_analysis['Mean'] = df_all_subjects.mean()
df_analysis['Std'] = df_all_subjects.std()
df_analysis['Var'] = df_all_subjects.var()
df_analysis

In [ ]:
# Saving the detected bursts for every muscle 
sr = 1000 # sample rate = 1000Hz
sl = 20 # smooth level (Size of sliding window used during the moving average process) #used to be 40
th = 10 # threshold (To cover activation)

# Initializing lists
detected_bursts_right_TA = [] ; detected_bursts_left_TA = []
detected_bursts_right_MG = [] ; detected_bursts_left_MG = []
detected_bursts_right_SOL = []; detected_bursts_left_SOL = []
detected_bursts_right_BF = [] ; detected_bursts_left_BF = []
detected_bursts_right_ST = [] ; detected_bursts_left_ST = []
detected_bursts_right_VL = [] ; detected_bursts_left_VL = []
detected_bursts_right_RF = [] ; detected_bursts_left_RF = []


pbar = tqdm(total=len(muscles)*len(list_of_all_subjects_dfs), desc="All Subjects Burst Detection Progress", unit= "Muscle")
for df_subject in list_of_all_subjects_dfs: ################# !!!!!!!!!!!!!!!!!!!!!!!!!!
    ## TA
    detected_bursts_right_TA.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_TA'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)  # Update progress bar for Right_TA
    detected_bursts_left_TA.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_TA'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)  
    ## MG
    detected_bursts_right_MG.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_MG'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)  
    detected_bursts_left_MG.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_MG'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)  
    ## SOL
    detected_bursts_right_SOL.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_SOL'], sample_rate=sr, smooth_level=sl,
                                                            threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)  
    detected_bursts_left_SOL.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_SOL'], sample_rate=sr, smooth_level=sl,
                                                            threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    ## BF
    detected_bursts_right_BF.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_BF'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    detected_bursts_left_BF.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_BF'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    ## ST
    detected_bursts_right_ST.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_ST'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    detected_bursts_left_ST.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_ST'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    ## VL
    detected_bursts_right_VL.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_VL'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    detected_bursts_left_VL.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_VL'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    ## RF
    detected_bursts_right_RF.append(bsnb.detect_emg_activations(emg_signal=df_subject['Right_RF'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)
    detected_bursts_left_RF.append(bsnb.detect_emg_activations(emg_signal=df_subject['Left_RF'], sample_rate=sr, smooth_level=sl,
                                                           threshold_level=th, time_units=True, device='CH0', plot_result= False))
    pbar.update(1)

In [ ]:
# # Visualising the EMG Burst Detection for the right Tibialis Anterior
# plot_duration = 40000 # time in milliseconds
# bsnb.detect_emg_activations(emg_signal = df_all_subjects['Right_TA'][:plot_duration], sample_rate = sr, smooth_level=sl, threshold_level=th, time_units=True, device='CH0', plot_result= True)
# print('')

In [ ]:
pd.DataFrame(detected_bursts_right_TA[0:3][:10]).transpose()

In [ ]:
# # Visualizing First Activations -> SEE CAPTURED WINDOW WITH RESPECT OF IDENTIFIED ACTIVATION
# duration = 8000
# shift = 2000
# number_bursts_to_plot = 1
# 
# plt.rcParams["figure.figsize"] = (10,5)
# fig = plt.figure()
# 
# plt.plot(df_all_subjects['Right_TA'][:duration], color="cornflowerblue")
# for i in range(number_bursts_to_plot): # Plot first N bursts
#     plt.axvline(detected_bursts_right_TA[0][i]*1000,color='red', label="Detected Burst Region") # ONSET VERTICAL LINE
#     plt.axvline(detected_bursts_right_TA[1][i]*1000,color='red') # OFFSET VERTICAL LINE
#     plt.axvline(detected_bursts_right_TA[0][i]*1000+400,color='black', label="Onset Window (500ms)") # ONSET VERTICAL LINE CORRECTED (START WINDOW)
#     plt.axvline(detected_bursts_right_TA[0][i]*1000-100,color='black') # VERTICAL LINE (END WINDOW)
#     
# plt.legend(loc="upper left")
# plt.xlim(shift,duration)
# plt.grid()
# plt.xlabel('Time (ms)', fontsize=10)
# plt.ylabel('sEMG Intensity (V)', fontsize=10)
# 
# # plt.savefig("Window.png")

In [ ]:
# # adding figures and traces
# fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x = df_all_subjects.index/1000 , y=df_all_subjects['Left_TA'][:10000]))
# 
# # formatting the plot
# fig1.update_layout(autosize=True, title="sEMG Signal: Detected burst and corrected onset window",
#                    xaxis_title="Time (s)", yaxis_title="sEMG Activity (V)", margin=dict(l=50, r=50, b=50, t=50, pad=4))
# 
# fig1.add_vrect(x0=detected_bursts_right_TA[0][0], x1=detected_bursts_left_TA[1][0], row="all", col=1,
#                annotation_text="Detected Burst", annotation_position="top right", fillcolor="gray",
#                opacity=0.25, line_width=0)
# 
# fig1.add_vline(x=detected_bursts_left_TA[0][0]+0.4,line_width=1.5, line_dash="dot", line_color="red")
# fig1.add_vline(x=detected_bursts_left_TA[0][0]-0.1,line_width=1.5, line_dash="dot", line_color="red",
#                annotation_text="Onset Window",annotation_position="bottom right")
# 
# # fig1.update_xaxes(range=[7.5, 20000/1000])
# # fig1.update_yaxes(range=[-2, 2])
# fig1.update_layout(autosize=False, width=800, height=300)
# # plotting
# fig1.show()

In [ ]:
# # adding figures and traces
# fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x= df_all_subjects.index/1000, y=df_all_subjects['Right_TA']))
# # formatting the plot
# fig1.update_layout(autosize=True, title="sEMG Signal: Detection of Activation Bursts",
#                    xaxis_title="Time (s)", yaxis_title="sEMG Activity (V)",
#                    margin=dict(l=50, r=50, b=50, t=50, pad=4))
# 
# for i in range(len(detected_bursts_right_TA[0])):
#     fig1.add_vrect(x0=detected_bursts_right_TA[0][i], x1=detected_bursts_right_TA[1][i], row="all", col=1,
#                    annotation_text="Detected Burst", annotation_position="top right",
#                    fillcolor="black", opacity=0.25, line_width=0)
# 
# # fig1.update_xaxes(range=[30, 60])
# fig1.update_layout(autosize=False, width=800, height=300)
# # plotting
# fig1.show()

In [ ]:
# # adding figures and traces
# fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x= df_all_subjects.index/1000, y=df_all_subjects['Left_ST']))
# # formatting the plot
# fig1.update_layout(autosize=True, title="sEMG Signal: Detection of Activation Bursts",
#                    xaxis_title="Time (s)", yaxis_title="sEMG Activity (V)",
#                    margin=dict(l=50, r=50, b=50, t=50, pad=4))
# 
# for i in range(len(detected_bursts_left_ST[0])):
#     fig1.add_vrect(x0=detected_bursts_left_ST[0][i], x1=detected_bursts_left_ST[1][i], row="all", col=1,
#                    annotation_text="Detected Burst", annotation_position="top right",
#                    fillcolor="black", opacity=0.25, line_width=0)
# 
# # fig1.update_xaxes(range=[30, 60])
# fig1.update_layout(autosize=False, width=800, height=300)
# # plotting
# fig1.show()

In [ ]:
# Calculating the total number of bursts per muscle
tot_bursts_right_TA = len(detected_bursts_right_TA[0][0]); tot_bursts_left_TA = len(detected_bursts_left_TA[0][0])
tot_bursts_right_MG = len(detected_bursts_right_MG[0][0]); tot_bursts_left_MG = len(detected_bursts_left_MG[0][0])
tot_bursts_right_SOL= len(detected_bursts_right_SOL[0][0]); tot_bursts_left_SOL= len(detected_bursts_left_SOL[0][0])
tot_bursts_right_BF = len(detected_bursts_right_BF[0][0]); tot_bursts_left_BF = len(detected_bursts_left_BF[0][0])
tot_bursts_right_ST = len(detected_bursts_right_ST[0][0]); tot_bursts_left_ST = len(detected_bursts_left_ST[0][0])
tot_bursts_right_VL = len(detected_bursts_right_VL[0][0]); tot_bursts_left_VL = len(detected_bursts_left_VL[0][0])
tot_bursts_right_RF = len(detected_bursts_right_RF[0][0]); tot_bursts_left_RF = len(detected_bursts_left_RF[0][0])

In [ ]:
# Printing the total number of bursts per muscle
print("- Number of Identified Bursts:\n",
      "   Right TA:\t", tot_bursts_right_TA, "\t", "   Left TA:\t", tot_bursts_left_TA,"\n",
      "   Right MG:\t", tot_bursts_right_MG, "\t", "   Left MG:\t", tot_bursts_left_MG,"\n",
      "   Right SOL:\t", tot_bursts_right_SOL, "\t", "   Left SOL:\t", tot_bursts_left_SOL,"\n",
      "   Right BF:\t", tot_bursts_right_BF, "\t", "   Left BF:\t", tot_bursts_left_BF,"\n",
      "   Right ST:\t", tot_bursts_right_ST, "\t", "   Left ST:\t", tot_bursts_left_ST,"\n",
      "   Right VL:\t", tot_bursts_right_VL, "\t", "   Left VL:\t", tot_bursts_left_VL,"\n",
      "   Right RF:\t", tot_bursts_right_RF, "\t", "   Left RF:\t", tot_bursts_left_RF,"\n")

In [ ]:
# Lists to store average burst length per muscle for all subjects
average_burst_lengths_right_TA = [] ; average_burst_lengths_left_TA = []
average_burst_lengths_right_MG = [] ; average_burst_lengths_left_MG = []
average_burst_lengths_right_SOL = []; average_burst_lengths_left_SOL = []
average_burst_lengths_right_BF = [] ; average_burst_lengths_left_BF = []
average_burst_lengths_right_ST = [] ; average_burst_lengths_left_ST = []
average_burst_lengths_right_VL = [] ; average_burst_lengths_left_VL = []
average_burst_lengths_right_RF = [] ; average_burst_lengths_left_RF = []

# Calculating average burst length per muscle
for subject in range(len(list_of_all_subjects_dfs)):
    ## TA
    average_burst_lengths_right_TA.append(np.mean(np.array(detected_bursts_right_TA[subject][1])-np.array(detected_bursts_right_TA[subject][0]))*1000)
    average_burst_lengths_left_TA.append(np.mean(np.array(detected_bursts_left_TA[subject][1])-np.array(detected_bursts_left_TA[subject][0]))*1000)
    ## MG
    average_burst_lengths_right_MG.append(np.mean(np.array(detected_bursts_right_MG[subject][1])-np.array(detected_bursts_right_MG[subject][0]))*1000)
    average_burst_lengths_left_MG.append(np.mean(np.array(detected_bursts_left_MG[subject][1])-np.array(detected_bursts_left_MG[subject][0]))*1000)
    ## SOL
    average_burst_lengths_right_SOL.append(np.mean(np.array(detected_bursts_right_SOL[subject][1])-np.array(detected_bursts_right_SOL[subject][0]))*1000)
    average_burst_lengths_left_SOL.append(np.mean(np.array(detected_bursts_left_SOL[subject][1])-np.array(detected_bursts_left_SOL[subject][0]))*1000)
    ## BF
    average_burst_lengths_right_BF.append(np.mean(np.array(detected_bursts_right_BF[subject][1])-np.array(detected_bursts_right_BF[subject][0]))*1000)
    average_burst_lengths_left_BF.append(np.mean(np.array(detected_bursts_left_BF[subject][1])-np.array(detected_bursts_left_BF[subject][0]))*1000)
    ## ST
    average_burst_lengths_right_ST.append(np.mean(np.array(detected_bursts_right_ST[subject][1])-np.array(detected_bursts_right_ST[subject][0]))*1000)
    average_burst_lengths_left_ST.append(np.mean(np.array(detected_bursts_left_ST[subject][1])-np.array(detected_bursts_left_ST[subject][0]))*1000)
    ## VL
    average_burst_lengths_right_VL.append(np.mean(np.array(detected_bursts_right_VL[subject][1])-np.array(detected_bursts_right_VL[subject][0]))*1000)
    average_burst_lengths_left_VL.append(np.mean(np.array(detected_bursts_left_VL[subject][1])-np.array(detected_bursts_left_VL[subject][0]))*1000)
    ## RF
    average_burst_lengths_right_RF.append(np.mean(np.array(detected_bursts_right_RF[subject][1])-np.array(detected_bursts_right_RF[subject][0]))*1000)
    average_burst_lengths_left_RF.append(np.mean(np.array(detected_bursts_left_RF[subject][1])-np.array(detected_bursts_left_RF[subject][0]))*1000)

In [ ]:
# Printing average burst length per muscle
print("- Avg. Length:\n",
      "   Right TA:\t", round(np.mean(average_burst_lengths_right_TA),2), "ms" , "\t", "   Left TA:\t", round(np.mean(average_burst_lengths_left_TA),2),"ms", "\n",
      "   Right MG:\t", round(np.mean(average_burst_lengths_right_MG), 2), "ms" , "\t", "   Left MG:\t", round(np.mean(average_burst_lengths_left_MG), 2), "ms", "\n",
      "   Right SOL:\t", round(np.mean(average_burst_lengths_right_SOL), 2), "ms" , "\t", "   Left SOL:\t", round(np.mean(average_burst_lengths_left_SOL), 2), "ms", "\n",
      "   Right BF:\t", round(np.mean(average_burst_lengths_right_BF), 2), "ms" , "\t", "   Left BF:\t", round(np.mean(average_burst_lengths_left_BF), 2), "ms", "\n",
      "   Right ST:\t", round(np.mean(average_burst_lengths_right_ST), 2), "ms" , "\t", "   Left ST:\t", round(np.mean(average_burst_lengths_left_ST), 2), "ms", "\n",
      "   Right VL:\t", round(np.mean(average_burst_lengths_right_VL), 2), "ms" , "\t", "   Left VL:\t", round(np.mean(average_burst_lengths_left_VL), 2), "ms", "\n",
      "   Right RF:\t", round(np.mean(average_burst_lengths_right_RF), 2), "ms" , "\t", "   Left RF:\t", round(np.mean(average_burst_lengths_left_RF), 2), "ms", "\n")


In [ ]:
# # Adjust subplot indexing
# f, a = plt.subplots(7, 2)
# f.set_size_inches(15, 40)
# a = a.ravel()
# 
# colors = plt.cm.tab10(np.linspace(0, 1, len(subjects)))
# 
# for subject_idx, subject in enumerate(subjects):
#     subject = "subject_"+subject
#     a[0].hist(np.array(detected_bursts_right_TA[subject_idx][1])-np.array(detected_bursts_right_TA[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[1].hist(np.array(detected_bursts_left_TA[subject_idx][1])-np.array(detected_bursts_left_TA[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx])
# 
#     a[2].hist(np.array(detected_bursts_right_MG[subject_idx][1])-np.array(detected_bursts_right_MG[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[3].hist(np.array(detected_bursts_left_MG[subject_idx][1])-np.array(detected_bursts_left_MG[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx])
# 
#     a[4].hist(np.array(detected_bursts_right_SOL[subject_idx][1])-np.array(detected_bursts_right_SOL[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[5].hist(np.array(detected_bursts_left_SOL[subject_idx][1])-np.array(detected_bursts_left_SOL[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx], density= True)
# 
#     a[6].hist(np.array(detected_bursts_right_BF[subject_idx][1])-np.array(detected_bursts_right_BF[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[7].hist(np.array(detected_bursts_left_BF[subject_idx][1])-np.array(detected_bursts_left_BF[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx])
# 
#     a[8].hist(np.array(detected_bursts_right_ST[subject_idx][1])-np.array(detected_bursts_right_ST[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[9].hist(np.array(detected_bursts_left_ST[subject_idx][1])-np.array(detected_bursts_left_ST[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx])
# 
#     a[10].hist(np.array(detected_bursts_right_VL[subject_idx][1])-np.array(detected_bursts_right_VL[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[11].hist(np.array(detected_bursts_left_VL[subject_idx][1])-np.array(detected_bursts_left_VL[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx])
# 
#     a[12].hist(np.array(detected_bursts_right_RF[subject_idx][1])-np.array(detected_bursts_right_RF[subject_idx][0]), bins=40, alpha=0.5, label=subject + " right", color=colors[subject_idx])
#     a[13].hist(np.array(detected_bursts_left_RF[subject_idx][1])-np.array(detected_bursts_left_RF[subject_idx][0]), bins=40, alpha=0.5, label=subject + " left", color=colors[subject_idx])
#     
#     # Repeat similar histograms for other muscles
# 
# for muscle_idx, muscle in enumerate(muscles):
#     # Set legends for each subplot
#     a[muscle_idx].legend(loc='upper right')
# 
#     # Set titles for each subplot
#     a[muscle_idx].set_title('Histogram Burst Duration: ' + muscle)
# 
#     # Set x and y labels for each subplot
#     a[muscle_idx].set_xlabel("Burst Duration (seconds)")
#     a[muscle_idx].set_ylabel("Occurrences")
#     # a[muscle_idx].
# 
# plt.tight_layout()
# plt.show()


In [ ]:
# TODO :  REDO THIS MY WAY 
# Longest muscle bursts 
muscles = ['TA', 'MG', 'SOL', 'BF', 'ST', 'VL', 'RF']
sides = ['right', 'left']

longest_pulses = {}

for muscle in muscles:
    for side in sides:
        bursts_variable_name = f"detected_bursts_{side}_{muscle}"
        bursts = eval(bursts_variable_name)  # Use with caution; assumes variable names follow a consistent naming convention
        
        longest_pulse = 0  # Initialize with zero to handle empty data
        
        for i in range(len(bursts)):
            pulse_durations = np.array(bursts[i][1]) - np.array(bursts[i][0])
            current_longest_pulse = max(pulse_durations, default=0)  # Use default=0 to handle empty arrays
            if current_longest_pulse > longest_pulse:
                longest_pulse = current_longest_pulse
                
        longest_pulses[f"{side}_{muscle}"] = longest_pulse

# Print the longest pulse for each muscle and side
for key, value in longest_pulses.items():
    print(f"Longest pulse for {key}:\t {round(value,2)} seconds")


## Leave one out: Separating a Subject

In [ ]:
leave_one_out = 1 # Specify which one to leave out, can also be None

In [ ]:
# if leave_one_out is not None:
loo_detected_bursts_right_TA = [detected_bursts_right_TA.pop(leave_one_out)]
loo_detected_bursts_left_TA  = [detected_bursts_left_TA.pop(leave_one_out)]
loo_detected_bursts_right_MG = [detected_bursts_right_MG.pop(leave_one_out)]
loo_detected_bursts_left_MG  = [detected_bursts_left_MG.pop(leave_one_out)]
loo_detected_bursts_right_SOL= [detected_bursts_right_SOL.pop(leave_one_out)]
loo_detected_bursts_left_SOL = [detected_bursts_left_SOL.pop(leave_one_out)]    
loo_detected_bursts_right_BF = [detected_bursts_right_BF.pop(leave_one_out)]
loo_detected_bursts_left_BF  = [detected_bursts_left_BF.pop(leave_one_out)]
loo_detected_bursts_right_ST = [detected_bursts_right_ST.pop(leave_one_out)]
loo_detected_bursts_left_ST  = [detected_bursts_left_ST.pop(leave_one_out)]
loo_detected_bursts_right_VL = [detected_bursts_right_VL.pop(leave_one_out)]
loo_detected_bursts_left_VL  = [detected_bursts_left_VL.pop(leave_one_out)]
loo_detected_bursts_right_RF = [detected_bursts_right_RF.pop(leave_one_out)]
loo_detected_bursts_left_RF  = [detected_bursts_left_RF.pop(leave_one_out)]

In [ ]:
# list_of_all_subjects_dfs.pop(leave_one_out)
# df_all_subjects = pd.concat(list_of_all_subjects_dfs, ignore_index=True)

In [ ]:
loo_emg_signal_right_TA  = list(list_of_all_subjects_dfs[leave_one_out]['Right_TA'])
loo_emg_signal_left_TA   = list(list_of_all_subjects_dfs[leave_one_out]['Left_TA'])
loo_emg_signal_right_MG  = list(list_of_all_subjects_dfs[leave_one_out]['Right_MG'])
loo_emg_signal_left_MG   = list(list_of_all_subjects_dfs[leave_one_out]['Left_MG'])
loo_emg_signal_right_SOL = list(list_of_all_subjects_dfs[leave_one_out]['Right_SOL'])
loo_emg_signal_left_SOL  = list(list_of_all_subjects_dfs[leave_one_out]['Left_SOL'])
loo_emg_signal_right_BF  = list(list_of_all_subjects_dfs[leave_one_out]['Right_BF'])
loo_emg_signal_left_BF   = list(list_of_all_subjects_dfs[leave_one_out]['Left_BF'])
loo_emg_signal_right_ST  = list(list_of_all_subjects_dfs[leave_one_out]['Right_ST'])
loo_emg_signal_left_ST   = list(list_of_all_subjects_dfs[leave_one_out]['Left_ST'])
loo_emg_signal_right_VL  = list(list_of_all_subjects_dfs[leave_one_out]['Right_VL'])
loo_emg_signal_left_VL   = list(list_of_all_subjects_dfs[leave_one_out]['Left_VL'])
loo_emg_signal_right_RF  = list(list_of_all_subjects_dfs[leave_one_out]['Right_RF'])
loo_emg_signal_left_RF   = list(list_of_all_subjects_dfs[leave_one_out]['Left_RF'])
print(f"{subjects[leave_one_out]} was excluded from the training!")

In [ ]:
# loo_emg_signal_right_TA

# Extracting Bursts

### Method: Preserving Onset and Window = 300ms

In [ ]:
def extract_burst_windows(muscle_emg_signal, onset_list, window_size, left_shift_value, muscle_name):
    sampling_rate = 1000 
    all_subjects_burst_samples = []
    for subject_index in tqdm(range(len(onset_list)), desc=f"Extracting Bursts for {muscle_name}"):
        current_subject_bursts = []
        ii = -1
        for onset in onset_list[subject_index][0]:
            ii += 1
            onset_ms = int(onset * sampling_rate) - left_shift_value
            current_sample_window = []
            if onset != onset_list[subject_index][0][-1]: 
                if (onset_ms + window_size) < (onset_list[subject_index][0][ii + 1] * 1000):
                    for time_step in range(window_size):
                        current_sample_window.append(muscle_emg_signal[onset_ms + time_step])
            else: 
                for time_step in range(window_size):
                    current_sample_window.append(muscle_emg_signal[onset_ms + time_step])

            if current_sample_window:
                current_subject_bursts.append(current_sample_window)
        all_subjects_burst_samples.append(current_subject_bursts)
    return all_subjects_burst_samples

In [ ]:
window = 300 # in ms (total window size)
left_shift = 50 # in ms (left shift from detected onset). See detected onset on the vertical red lines in the plots above

fixed_bursts_right_TA = extract_burst_windows(df_all_subjects['Right_TA'], detected_bursts_right_TA, window, left_shift, 'Right TA')
fixed_bursts_left_TA = extract_burst_windows(df_all_subjects['Left_TA'], detected_bursts_left_TA, window, left_shift, 'Left TA')
fixed_bursts_right_MG = extract_burst_windows(df_all_subjects['Right_MG'], detected_bursts_right_MG, window, left_shift, 'Right MG')
fixed_bursts_left_MG = extract_burst_windows(df_all_subjects['Left_MG'], detected_bursts_left_MG, window, left_shift, 'Left MG')
fixed_bursts_right_SOL = extract_burst_windows(df_all_subjects['Right_SOL'], detected_bursts_right_SOL, window, left_shift, 'Right SOL')
fixed_bursts_left_SOL = extract_burst_windows(df_all_subjects['Left_SOL'], detected_bursts_left_SOL, window, left_shift, 'Left SOL')
fixed_bursts_right_BF = extract_burst_windows(df_all_subjects['Right_BF'], detected_bursts_right_BF, window, left_shift, 'Right BF')
fixed_bursts_left_BF = extract_burst_windows(df_all_subjects['Left_BF'], detected_bursts_left_BF, window, left_shift, 'Left BF')
fixed_bursts_right_ST = extract_burst_windows(df_all_subjects['Right_ST'], detected_bursts_right_ST, window, left_shift, 'Right ST')
fixed_bursts_left_ST = extract_burst_windows(df_all_subjects['Left_ST'], detected_bursts_left_ST, window, left_shift, 'Left ST')
fixed_bursts_right_VL = extract_burst_windows(df_all_subjects['Right_VL'], detected_bursts_right_VL, window, left_shift, 'Right VL')
fixed_bursts_left_VL = extract_burst_windows(df_all_subjects['Left_VL'], detected_bursts_left_VL, window, left_shift, 'Left VL')
fixed_bursts_right_RF = extract_burst_windows(df_all_subjects['Right_RF'], detected_bursts_right_RF, window, left_shift, 'Right RF')
fixed_bursts_left_RF = extract_burst_windows(df_all_subjects['Left_RF'], detected_bursts_left_RF, window, left_shift, 'Left RF')

In [ ]:
print("Number of Muscle Bursts Per Subject (Fixed Window):\n",
      "   Right TA:\t", [(len(lst),) for lst in fixed_bursts_right_TA], "\n", 
      "   Left TA:\t", [(len(lst),) for lst in fixed_bursts_left_TA], "\n",
      "   Right MG:\t", [(len(lst),) for lst in fixed_bursts_right_MG], "\n", 
      "   Left MG:\t", [(len(lst),) for lst in fixed_bursts_left_MG], "\n",
      "   Right SOL:\t", [(len(lst),) for lst in fixed_bursts_right_SOL], "\n", 
      "   Left SOL:\t", [(len(lst),) for lst in fixed_bursts_left_SOL], "\n",
      "   Right BF:\t", [(len(lst),) for lst in fixed_bursts_right_BF], "\n", 
      "   Left BF:\t", [(len(lst),) for lst in fixed_bursts_left_BF], "\n",
      "   Right ST:\t", [(len(lst),) for lst in fixed_bursts_right_ST], "\n", 
      "   Left ST:\t", [(len(lst),) for lst in fixed_bursts_left_ST], "\n",
      "   Right VL:\t", [(len(lst),) for lst in fixed_bursts_right_VL], "\n", 
      "   Left VL:\t", [(len(lst),) for lst in fixed_bursts_left_VL], "\n",
      "   Right RF:\t", [(len(lst),) for lst in fixed_bursts_right_RF], "\n", 
      "   Left RF:\t", [(len(lst),) for lst in fixed_bursts_left_RF], "\n")


In [ ]:
# if leave_one_out is not None:
loo_fixed_bursts_right_TA = extract_burst_windows(loo_emg_signal_right_TA, loo_detected_bursts_right_TA, window, left_shift, 'Leave one Out: Right TA')[0]
loo_fixed_bursts_left_TA = extract_burst_windows(loo_emg_signal_left_TA, loo_detected_bursts_left_TA, window, left_shift, 'Leave one Out: Left TA')[0]
loo_fixed_bursts_right_MG = extract_burst_windows(loo_emg_signal_right_MG, loo_detected_bursts_right_MG, window, left_shift, 'Leave one Out: Right MG')[0]
loo_fixed_bursts_left_MG = extract_burst_windows(loo_emg_signal_left_MG, loo_detected_bursts_left_MG, window, left_shift, 'Leave one Out: Left MG')[0]
loo_fixed_bursts_right_SOL = extract_burst_windows(loo_emg_signal_right_SOL, loo_detected_bursts_right_SOL, window, left_shift, 'Leave one Out: Right SOL')[0]
loo_fixed_bursts_left_SOL = extract_burst_windows(loo_emg_signal_left_SOL, loo_detected_bursts_left_SOL, window, left_shift, 'Leave one Out: Left SOL')[0]
loo_fixed_bursts_right_BF = extract_burst_windows(loo_emg_signal_right_BF, loo_detected_bursts_right_BF, window, left_shift, 'Leave one Out: Right BF')[0]
loo_fixed_bursts_left_BF = extract_burst_windows(loo_emg_signal_left_BF, loo_detected_bursts_left_BF, window, left_shift, 'Leave one Out: Left BF')[0]
loo_fixed_bursts_right_ST = extract_burst_windows(loo_emg_signal_right_ST, loo_detected_bursts_right_ST, window, left_shift, 'Leave one Out: Right ST')[0]
loo_fixed_bursts_left_ST = extract_burst_windows(loo_emg_signal_left_ST, loo_detected_bursts_left_ST, window, left_shift, 'Leave one Out: Left ST')[0]
loo_fixed_bursts_right_VL = extract_burst_windows(loo_emg_signal_right_VL, loo_detected_bursts_right_VL, window, left_shift, 'Leave one Out: Right VL')[0]
loo_fixed_bursts_left_VL = extract_burst_windows(loo_emg_signal_left_VL, loo_detected_bursts_left_VL, window, left_shift, 'Leave one Out: Left VL')[0]
loo_fixed_bursts_right_RF = extract_burst_windows(loo_emg_signal_right_RF, loo_detected_bursts_right_RF, window, left_shift, 'Leave one Out: Right RF')[0]
loo_fixed_bursts_left_RF = extract_burst_windows(loo_emg_signal_left_RF, loo_detected_bursts_left_RF, window, left_shift, 'Leave one Out: Left RF')[0]

In [ ]:
# if leave_one_out != None:
print("------------------------\nLeave One Out: Number of Muscle Bursts (Fixed Window):\n",
      "   Right TA:\t", len(loo_fixed_bursts_right_TA), "\t", "   Left TA:\t", len(loo_fixed_bursts_left_TA), "\n",
      "   Right MG:\t", len(loo_fixed_bursts_right_MG), "\t", "   Left MG:\t", len(loo_fixed_bursts_left_MG), "\n",
      "   Right SOL:\t", len(loo_fixed_bursts_right_SOL), "\t", "   Left SOL:\t", len(loo_fixed_bursts_left_SOL), "\n",
      "   Right BF:\t", len(loo_fixed_bursts_right_BF), "\t", "   Left BF:\t", len(loo_fixed_bursts_left_BF), "\n",
      "   Right ST:\t", len(loo_fixed_bursts_right_ST), "\t", "   Left ST:\t", len(loo_fixed_bursts_left_ST), "\n",
      "   Right VL:\t", len(loo_fixed_bursts_right_VL), "\t", "   Left VL:\t", len(loo_fixed_bursts_left_VL), "\n",
      "   Right RF:\t", len(loo_fixed_bursts_right_RF), "\t", "   Left RF:\t", len(loo_fixed_bursts_left_RF))


## Saving Leave One Out as TFRecord

In [ ]:
loo_emg_series_complete = [
    loo_fixed_bursts_right_TA, loo_fixed_bursts_left_TA, 
    loo_fixed_bursts_right_MG, loo_fixed_bursts_left_MG, 
    loo_fixed_bursts_right_SOL, loo_fixed_bursts_left_SOL, 
    loo_fixed_bursts_right_BF, loo_fixed_bursts_left_BF, 
    loo_fixed_bursts_right_ST, loo_fixed_bursts_left_ST, 
    loo_fixed_bursts_right_VL, loo_fixed_bursts_left_VL, 
    loo_fixed_bursts_right_RF, loo_fixed_bursts_left_RF]

muscle_groups = len(loo_emg_series_complete)

with tf.io.TFRecordWriter('leave_one_out.tfrecord') as tfrecord:
    for emg_muscle in tqdm(range(muscle_groups), desc="Extracting dataset to TFRecords", position=0, leave=True):
        for sample in loo_emg_series_complete[emg_muscle]:
            # Prepare the features for TFRecord
            features = {
                'label': tf.train.Feature(float_list=tf.train.FloatList(value=tf.keras.utils.to_categorical(emg_muscle, 14))),
                'feature': tf.train.Feature(float_list=tf.train.FloatList(value=sample)),
            }
            example = tf.train.Example(features=tf.train.Features(feature=features))
            tfrecord.write(example.SerializeToString())

In [ ]:
# Iterate over the whole dataset to count records/samples (https://www.rustyrobotics.com/posts/tensorflow/tfdataset-record-count/)
# Reference: https://www.rustyrobotics.com/posts/tensorflow/tfdataset-record-count/
def countRecords(ds:tf.data.Dataset):
  count = 0
  if tf.executing_eagerly():
    # TF v2 or v1 in eager mode
    for _ in ds:
      count = count+1
  else:  
    # TF v1 in non-eager mode
    iterator = tf.compat.v1.data.make_one_shot_iterator(ds)
    next_batch = iterator.get_next()
    with tf.compat.v1.Session() as sess:
      try:
        while True:
          sess.run(next_batch)
          count = count+1    
      except tf.errors.OutOfRangeError:
        pass
  return count

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 1024
window = 300

def loo_read_tfrecord(serialized_example):
    tfrecord_format = (
        {
            'label': tf.io.FixedLenFeature([14], tf.float32),  # MUSCLE LABEL: Adjusted for one-hot encoded labels
            'feature': tf.io.FixedLenFeature([300], tf.float32),  # sEMG VALUE: Adjusted for features. CHANGE 300 LATER ON FOR TRAINING NEEDS!!!!!!!!!!!!!!!!!!!!!!!!!!
        }
    )
    example = tf.io.parse_single_example(serialized_example, tfrecord_format)
    f = tf.reshape(example['feature'], [window, 1])  # Reshape if needed, here it's kept for consistency
    f.set_shape([window, 1])
    return f, example['label']

def loo_get_dataset(tf_record_name):
    dataset = tf.data.TFRecordDataset(tf_record_name)
    dataset = dataset.map(loo_read_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset_samples = countRecords(dataset)  
    dataset = dataset.shuffle(dataset_samples)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

loo_dataset = loo_get_dataset('leave_one_out.tfrecord')

In [ ]:
for feature, label in loo_dataset:
    print('label={}, feature={}'.format(label.shape,feature.shape))

# TFRecords: Storing Training and Validation Datasets in Tensorflow Records

Reference: https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/tfrecord.ipynb#scrollTo=_e3g9ExathXP

## Write TFRecords

In [ ]:
def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def extract_burst_windows_tfrecord(emg_series_complete, onset_lists, window_size, left_shift_value):
    # Ensure unique file names for parallel processing or repeated calls
    file_name = 'all_dataset.tfrecord'
    with tf.io.TFRecordWriter(file_name) as tfrecord:
        for muscle_index in tqdm(range(len(emg_series_complete)), desc="Extracting dataset to TFRecords (Fixed Window)"):
            for subject_index in range(len(onset_lists[muscle_index])):
                burst_count = 0
                ii = -1
                for onset in onset_lists[muscle_index][subject_index][0]:
                    ii += 1
                    onset_ms = int(onset * 1000) - left_shift_value
                    current_sample_window = []
                    if onset != onset_lists[muscle_index][subject_index][0][-1]: 
                        if (onset_ms + window_size) < (onset_lists[muscle_index][subject_index][0][ii + 1] * 1000):
                            burst_count = burst_count + 1
                            for time_step in range(window_size):
                                current_sample_window.append(emg_series_complete[muscle_index][onset_ms + time_step])
                    else: 
                        burst_count = burst_count + 1
                        for time_step in range(window_size):
                            current_sample_window.append(emg_series_complete[muscle_index][onset_ms + time_step])

                    if current_sample_window:
                        # Convert your sample and label to appropriate tf.train.Feature formats
                        features = {
                            'label': _int64_feature(muscle_index),
                            'feature': tf.train.Feature(float_list=tf.train.FloatList(value=current_sample_window)),
                            'subject': _int64_feature(subject_index + 1),
                            'burst': _int64_feature(burst_count),
                        }
                        example = tf.train.Example(features=tf.train.Features(feature=features))
                        tfrecord.write(example.SerializeToString())
                    # else:
                    #     print(f"Skipped a sample due to insufficient length. Muscle: {emg_muscle}, Subject: {subject_index}, Burst: {burst_count}")
    return file_name

In [ ]:
# for list_of_all_subjects_dfs
emg_series_tot = [df_all_subjects['Left_TA'], df_all_subjects['Right_TA'], df_all_subjects['Left_MG'], df_all_subjects['Right_MG'], df_all_subjects['Left_SOL'], df_all_subjects['Right_SOL'], df_all_subjects['Left_BF'], df_all_subjects['Right_BF'], df_all_subjects['Left_ST'], df_all_subjects['Right_ST'], df_all_subjects['Left_VL'], df_all_subjects['Right_VL'], df_all_subjects['Left_RF'], df_all_subjects['Right_RF']]

detected_bursts_tot = [detected_bursts_left_TA, detected_bursts_right_TA, detected_bursts_left_MG, detected_bursts_right_MG, detected_bursts_left_SOL, detected_bursts_right_SOL, detected_bursts_left_BF, detected_bursts_right_BF, detected_bursts_left_ST, detected_bursts_right_ST, detected_bursts_left_VL, detected_bursts_right_VL, detected_bursts_left_RF, detected_bursts_right_RF]

In [ ]:
# len(emg_series_tot[1])

In [ ]:
extract_burst_windows_tfrecord(emg_series_tot, detected_bursts_tot,  window, left_shift)

## Read TFRecords

In [ ]:
def map_fn(serialized_example):
    features = {
        'label': tf.io.FixedLenFeature([], tf.int64),
        'feature': tf.io.FixedLenFeature([300], tf.float32),
        'subject': tf.io.FixedLenFeature([], tf.int64),
        'burst':  tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(serialized_example, features)
    return example['label'], example['feature'], example['subject'], example['burst']

dataset = tf.data.TFRecordDataset('all_dataset.tfrecord')
dataset = dataset.map(map_fn)

for label, feature, subject, burst in dataset:
    print(f'label={label}, Number of features={len(feature)}  subject={subject}, burst={burst}')

# plt.plot(ch1)

In [ ]:
def separate_dataset_per_subject_train_val(dataset, subj, train_percentage):
    # Filtering whole dataset TFRECORDS by subjects:
    dataset_subject = dataset.filter(lambda label,feature,subject,burst: subject==subj)
    # Count Total Samples for each Subject Dataset
    dataset_subject_samples = countRecords(dataset_subject)
    # Shuffling bursts per subject
    dataset_subject_shuffled = dataset_subject.shuffle(dataset_subject_samples)
    # Separating Subject Training and Evaluation Datasets:
    dataset_subject_1_train = dataset_subject_shuffled.take(int(dataset_subject_samples*train_percentage))
    dataset_subject_1_val = dataset_subject_shuffled.skip(int(dataset_subject_samples*train_percentage)).take(dataset_subject_samples - int(dataset_subject_samples*train_percentage))
    return dataset_subject_1_train, dataset_subject_1_val

train_percentage = 0.8
dataset_subject1_train, dataset_subject1_val = separate_dataset_per_subject_train_val(dataset, 1, train_percentage)
dataset_subject2_train, dataset_subject2_val = separate_dataset_per_subject_train_val(dataset, 2, train_percentage)
# dataset_subject3_train, dataset_subject3_val = separate_dataset_per_subject_train_val(dataset, 3, train_percentage)
# dataset_subject4_train, dataset_subject4_val = separate_dataset_per_subject_train_val(dataset, 4, train_percentage)
# dataset_subject5_train, dataset_subject5_val = separate_dataset_per_subject_train_val(dataset, 5, train_percentage)
# dataset_subject6_train, dataset_subject6_val = separate_dataset_per_subject_train_val(dataset, 6, train_percentage)
# dataset_subject7_train, dataset_subject7_val = separate_dataset_per_subject_train_val(dataset, 7, train_percentage)
# dataset_subject8_train, dataset_subject8_val = separate_dataset_per_subject_train_val(dataset, 8, train_percentage)
# dataset_subject9_train, dataset_subject9_val = separate_dataset_per_subject_train_val(dataset, 9, train_percentage)
    
for label, feature, subject, burst in dataset_subject1_train.take(10):
    print(f'label={label}, Number of features={len(feature)}  subject={subject}, burst={burst}')
    
# print(countRecords(dataset_subject1_train))

In [ ]:
all_subject_datasets_train = [dataset_subject1_train,
                              dataset_subject2_train]#,
                              # dataset_subject3_train,
                              # dataset_subject4_train,
                              # dataset_subject5_train,
                              # dataset_subject6_train,
                              # dataset_subject7_train,
                              # dataset_subject8_train]

all_subject_datasets_val =   [dataset_subject1_val,
                              dataset_subject2_val] #,
                              # dataset_subject3_val,
                              # dataset_subject4_val,
                              # dataset_subject5_val,
                              # dataset_subject6_val,
                              # dataset_subject7_val,
                              # dataset_subject8_val]

def augment_datasets(collection_datasets, tf_record_name):
    with tf.io.TFRecordWriter(tf_record_name) as tfrecord:
        for d in collection_datasets:
            for label, feature, subject, burst in d:
                # print('label={}, subject={}, burst={}'.format(label,subject,burst))
                # feature = eval(eval('"ch" + str(i+1)'))
                features = {
                    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=np.asarray([label]))),
                    'feature': tf.train.Feature(float_list=tf.train.FloatList(value=np.asarray(feature))),
                    'subject': tf.train.Feature(int64_list=tf.train.Int64List(value=np.asarray([subject]))),
                    'burst': tf.train.Feature(int64_list=tf.train.Int64List(value=np.asarray([burst])))
                }
                example = tf.train.Example(features=tf.train.Features(feature=features))
                tfrecord.write(example.SerializeToString())
    return

def map_fn_final(serialized_example):
    features = {
        # Assuming label is a scalar, not a 14-element vector
        'label': tf.io.FixedLenFeature([], tf.int64),
        'feature': tf.io.FixedLenFeature([300], tf.float32),
        'subject': tf.io.FixedLenFeature([], tf.int64),
        'burst': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(serialized_example, features)
    return example['label'], example['feature'], example['subject'], example['burst']


augment_datasets(all_subject_datasets_train, 'all_mixed_train.tfrecord')
augment_datasets(all_subject_datasets_val, 'all_mixed_val.tfrecord')

def mix_and_shuffle_datasets(tf_record_name):
    dataset = tf.data.TFRecordDataset(tf_record_name)
    dataset = dataset.map(map_fn_final)
    dataset_samples = countRecords(dataset)
    dataset_final = dataset.shuffle(dataset_samples)
    return dataset_final
    
dataset_final_train = mix_and_shuffle_datasets('all_mixed_train.tfrecord')
dataset_final_val = mix_and_shuffle_datasets('all_mixed_val.tfrecord')

In [ ]:
for label, feature, subject, burst in dataset_final_train.take(20):
    print(f'label={label}, feature={len(feature)}, subject={subject}, burst={burst}')

## Load TFRecords

In [ ]:
def read_tfrecord(serialized_example, export_subject=False):
    tfrecord_format = (
        {
            'label': tf.io.FixedLenFeature([], tf.int64),
            'feature': tf.io.FixedLenFeature([300], tf.float32),
            'subject': tf.io.FixedLenFeature([], tf.int64),
            'burst':  tf.io.FixedLenFeature([], tf.int64),
        }
    )
    example = tf.io.parse_single_example(serialized_example, tfrecord_format)
    f = tf.reshape(example['feature'], [window,1])
    f.set_shape([window, 1])
    if export_subject:
        return f,example['label'],example['subject']
    return f,example['label']

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 1024
window = 300

def get_dataset(tf_record_name):
#     dataset = load_dataset(filename)
    dataset = tf.data.TFRecordDataset(tf_record_name)
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTOTUNE)
    dataset_samples = countRecords(dataset)
#     print("Samples: ", dataset_samples)
    dataset = dataset.shuffle(dataset_samples)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [ ]:
train_dataset = get_dataset('all_mixed_train.tfrecord')
valid_dataset = get_dataset('all_mixed_val.tfrecord')

In [ ]:
for feature, label in loo_dataset:
    print(f'label={label.shape}, feature={feature.shape}')

## Plotting Muscle Bursts

In [ ]:
# quick plot to see individual contraction bursts
def plot_independent_bursts(label, burst_list, color):
    number_plots = len(burst_list) # number of subjects
    n_cols = number_plots 
    # n_cols = 5
    fig, axs = plt.subplots(1, n_cols, figsize=(15,2) , dpi=150)
    fig.suptitle(f'Contraction Bursts: {label} (Fixed length)')
    n_burst = 0
    for j in range(n_cols):
        axs[j].plot(burst_list[j][n_burst], color=color)
        axs[j].set_title(f'1st Burst of subject {j+1}')
        n_burst+=1
        
    for ax in axs.flat:
      ax.set(xlabel='time (ms)', ylabel='EMG')
      #ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    #Hide x labels and tick labels for top plots and y ticks for right plots.
    for ax in axs.flat:
      ax.label_outer()

plot_independent_bursts("Left TA", fixed_bursts_left_TA, "cornflowerblue")
plot_independent_bursts("Right TA", fixed_bursts_right_TA, "yellowgreen")
plot_independent_bursts("Left MG", fixed_bursts_left_MG, "orange")
plot_independent_bursts("Right MG", fixed_bursts_right_MG, "purple")
plot_independent_bursts("Left SOL", fixed_bursts_left_SOL, "red")
plot_independent_bursts("Right SOL", fixed_bursts_right_SOL, "blue")
plot_independent_bursts("Left BF", fixed_bursts_left_BF, "green")
plot_independent_bursts("Right BF", fixed_bursts_right_BF, "yellow")
plot_independent_bursts("Left ST", fixed_bursts_left_ST, "orange")
plot_independent_bursts("Right ST", fixed_bursts_right_ST, "pink")
plot_independent_bursts("Left VL", fixed_bursts_left_VL, "cyan")
plot_independent_bursts("Right VL", fixed_bursts_right_VL, "magenta")
plot_independent_bursts("Left RF", fixed_bursts_left_RF, "brown")
plot_independent_bursts("Right RF", fixed_bursts_right_RF, "grey")

## GPU Test: